<a href="https://colab.research.google.com/github/ethvedbitdesjan/vedaant-jain-sectioning-info-extraction/blob/main/EmploymentIE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('content/')

Mounted at content/


In [ ]:
%cd /content/content/MyDrive/Resume_Info_Extraction/Academic_Appointments

/content/content/MyDrive/Resume_Info_Extraction/Academic_Appointments


In [ ]:
import json

In [ ]:
import pandas as pd

In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 17.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 65.7 MB/s eta 0:00

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 7.1 MB/s eta 0:00:00


In [ ]:
!pip install seqeval
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=4ca744846386c8ff4025e680c00528ef5232ab0ba2492f97d31be3856d5c342b
  Stored in directory: /root/.cache/pip/wheels/e2/a5/92/2c80d1928733611c2747a9820e1324a6835524d9411510c142
Successfully built seqeval
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

from transformers import AutoTokenizer

from tqdm import tqdm
import tensorflow as tf

import matplotlib.pyplot as plt

In [ ]:
def load_sentences(filepath):

    final = []
    sentences = []

    with open(filepath, 'r') as f:
        
        for line in f.readlines():
            
            if (line == ('-DOCSTART- -X- O\n') or line == '\n'):
                if len(sentences) > 0:
                    final.append(sentences)
                    sentences = []
            else:
                l = line.split(' ')
                sentences.append((l[0], l[3].strip('\n')))
    
    return final

In [ ]:
base_path = 'labelled_data.conll'

train_samples = load_sentences(base_path)

samples = train_samples

schema = sorted({tag for sentence in samples 
                             for _, tag in sentence})

In [ ]:
schema_id = {label:i for i, label in enumerate(schema)}
schema_id

{'B-Designation': 0,
 'B-Employer': 1,
 'I-Designation': 2,
 'I-Employer': 3,
 'O': 4}

In [ ]:
samples = [[np.array([samp[0], schema_id[samp[1]]]) for samp in sample] for sample in samples]

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "bert-large-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
samples= [np.array(sample) for sample in samples]
sample = samples[0]
samples_np = np.array(samples)
samples_np[0][:, 0]

<ipython-input-14-af730befd073>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  samples_np = np.array(samples)


array(['Academic', 'director', 'of', '|-Degrees', ',', '2019-',
       'Director', 'of', 'the', 'Finance', 'PhD', 'program', ',',
       '2008-2018', 'Professor', 'of', 'Finance', ',', '2009-2011',
       'Associate', 'Professor', 'of', 'Finance', ',', '2007-2009',
       'Research', 'Associate', ',', 'National', 'Bureau', 'of',
       'Economic', 'Research', ',', '2010-present', 'Faculty', 'Research',
       'Fellow', ';', 'National', 'Bureau', 'of', 'Economic', 'Research',
       ',', '2005-2010', 'New', 'York', 'University', ',', 'New', 'York',
       ',', 'NY', 'Associate', 'Professor', 'of', 'Finance', ',', 'Stern',
       'School', 'of', 'Business', ',', '2007-2008', 'Assistant',
       'Professor', 'of', 'Finance', ',', 'Stern', 'School', 'of',
       'Business', ',', '2000-2006', 'FGV-Rio', ',', 'Rio', 'de',
       'Janeiro', ',', 'Brazil', 'Visiting', 'Professor', ';',
       '2001-2003'], dtype='<U21')

In [ ]:
custom_dataset = [{"tokens":sample[:, 0], "tags":sample[:, 1].astype(int)} for sample in samples]

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples):
    word_tokens = [token for token in examples['tokens']]
    tokenized_inputs = tokenizer(
        word_tokens, truncation=True, is_split_into_words=True)
    
    all_labels = examples["tags"]
    new_labels = []
    word_ids = tokenized_inputs.word_ids()
    new_labels = align_labels_with_tokens(all_labels, word_ids)

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
samples[0][:, 1]

array(['0', '2', '2', '2', '4', '4', '0', '2', '2', '2', '2', '2', '4',
       '4', '0', '2', '2', '4', '4', '0', '2', '2', '2', '4', '4', '0',
       '2', '4', '1', '3', '3', '3', '3', '4', '4', '0', '2', '2', '4',
       '1', '3', '3', '3', '3', '4', '4', '1', '3', '3', '3', '3', '3',
       '3', '3', '0', '2', '2', '2', '4', '1', '3', '3', '3', '4', '4',
       '0', '2', '2', '2', '4', '1', '3', '3', '3', '4', '4', '1', '3',
       '3', '3', '3', '3', '3', '0', '2', '4', '4'], dtype='<U21')

In [ ]:
tokenized_dataset = [tokenize_and_align_labels(data) for data in custom_dataset]

In [ ]:
len(tokenized_dataset), tokenized_dataset[0]

(14,
 {'input_ids': [101, 10076, 1900, 1104, 197, 118, 16861, 1116, 117, 10351, 118, 2524, 1104, 1103, 7476, 7735, 1788, 117, 1369, 118, 1857, 2986, 1104, 7476, 117, 1371, 118, 1349, 9666, 2986, 1104, 7476, 117, 1384, 118, 1371, 2713, 9666, 117, 1305, 4447, 1104, 6051, 2713, 117, 1333, 118, 1675, 6694, 2713, 6229, 132, 1305, 4447, 1104, 6051, 2713, 117, 1478, 118, 1333, 1203, 1365, 1239, 117, 1203, 1365, 117, 5883, 9666, 2986, 1104, 7476, 117, 15010, 1323, 1104, 3518, 117, 1384, 118, 1369, 5414, 2986, 1104, 7476, 117, 15010, 1323, 1104, 3518, 117, 1539, 118, 1386, 143, 2349, 2559, 118, 5470, 117, 5470, 1260, 11502, 117, 3524, 19383, 2986, 132, 1630, 118, 1581, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_dataset[i] for i in range(2)])
batch["labels"]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


tensor([[-100,    0,    2,    2,    2,    2,    2,    2,    4,    4,    4,    0,
            2,    2,    2,    2,    2,    4,    4,    4,    4,    0,    2,    2,
            4,    4,    4,    4,    0,    2,    2,    2,    4,    4,    4,    4,
            0,    2,    4,    1,    3,    3,    3,    3,    4,    4,    4,    4,
            0,    2,    2,    4,    1,    3,    3,    3,    3,    4,    4,    4,
            4,    1,    3,    3,    3,    3,    3,    3,    3,    0,    2,    2,
            2,    4,    1,    3,    3,    3,    4,    4,    4,    4,    0,    2,
            2,    2,    4,    1,    3,    3,    3,    4,    4,    4,    4,    1,
            2,    2,    2,    2,    3,    3,    3,    3,    3,    3,    0,    2,
            4,    4,    4,    4, -100],
        [-100,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,    4,
            4,    4,    4,    1,    3,    3,    3,    3,    4,    4,    4,    4,
            4,    4,    0,    2,    2,    2,    4,    4,    0,    2, 

In [ ]:
for i in range(2):
    print(tokenized_dataset[i]["labels"])

[-100, 0, 2, 2, 2, 2, 2, 2, 4, 4, 4, 0, 2, 2, 2, 2, 2, 4, 4, 4, 4, 0, 2, 2, 4, 4, 4, 4, 0, 2, 2, 2, 4, 4, 4, 4, 0, 2, 4, 1, 3, 3, 3, 3, 4, 4, 4, 4, 0, 2, 2, 4, 1, 3, 3, 3, 3, 4, 4, 4, 4, 1, 3, 3, 3, 3, 3, 3, 3, 0, 2, 2, 2, 4, 1, 3, 3, 3, 4, 4, 4, 4, 0, 2, 2, 2, 4, 1, 3, 3, 3, 4, 4, 4, 4, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 0, 2, 4, 4, 4, 4, -100]
[-100, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 0, 2, 2, 2, 4, 4, 0, 2, 2, 2, 2, 2, 2, 4, 4, 0, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 1, 2, 3, 1, 2, 2, 2, 3, 3, 0, 2, 2, 2, 4, 4, 4, 1, 3, 3, 3, 0, 2, 2, 2, 4, 4, 4, 1, 2, 2, 1, 2, 0, 2, 2, 4, 4, 4, 1, 3, 3, 0, 2, 2, 2, 4, 4, 4, 0, 2, 4, 4, 4, -100]


In [ ]:
import evaluate

metric = evaluate.load("seqeval")

In [ ]:
label_names = list(schema_id.keys())
label_names

['B-Designation', 'B-Employer', 'I-Designation', 'I-Employer', 'O']

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {i: label for label, i in schema_id.items()}
id2label

{0: 'B-Designation',
 1: 'B-Employer',
 2: 'I-Designation',
 3: 'I-Employer',
 4: 'O'}

In [ ]:
from transformers import AutoModelForTokenClassification

In [ ]:
%mkdir bert_seq_tag
%cd /content/bert_seq_tag

[Errno 2] No such file or directory: '/content/bert_seq_tag'
/content/content/MyDrive/Resume_Info_Extraction/Academic_Appointments


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset[:2] + tokenized_dataset[4:],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=1,
)
eval_dataloader = DataLoader(
    tokenized_dataset[2:4], collate_fn=data_collator, batch_size=1
)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=schema_id,
)
model.config.num_labels

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-large-c

5

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:

from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 5
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
output_dir = "bert_seq_tag"

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_labels, true_predictions = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f"overall_{key}"]
            for key in ["precision", "recall", "f1", "accuracy"]
        },
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        

  0%|          | 0/60 [00:00<?, ?it/s]

epoch 0: {'precision': 0.08, 'recall': 0.058823529411764705, 'f1': 0.06779661016949153, 'accuracy': 0.7290836653386454}
epoch 1: {'precision': 0.34146341463414637, 'recall': 0.4117647058823529, 'f1': 0.37333333333333335, 'accuracy': 0.8247011952191236}
epoch 2: {'precision': 0.3409090909090909, 'recall': 0.4411764705882353, 'f1': 0.38461538461538464, 'accuracy': 0.8446215139442231}
epoch 3: {'precision': 0.4, 'recall': 0.47058823529411764, 'f1': 0.4324324324324324, 'accuracy': 0.8645418326693227}
epoch 4: {'precision': 0.43902439024390244, 'recall': 0.5294117647058824, 'f1': 0.48, 'accuracy': 0.8725099601593626}


In [ ]:
def map_token(a):
  if a not in id2label:
    return 'missing'
  return id2label[a]

In [ ]:
batch = data_collator([tokenized_dataset[3]])
batch["labels"]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch.to(device)
outputs = model(**batch)

In [ ]:
predictions = outputs.logits.argmax(dim=-1)
labels = batch["labels"]

In [ ]:
predictions.shape, labels.shape

(torch.Size([1, 53]), torch.Size([1, 53]))

In [ ]:
true_labels, true_predictions = postprocess(predictions, labels)

In [ ]:
schema_id['missing'] = 5

In [ ]:
list1 = [map_token(inp) for inp in predictions[0].cpu().detach().numpy()]
list2 = [map_token(inp) for inp in labels[0].cpu().detach().numpy()]
pred_idx = [i for i in range(len(list1)) if schema_id[list1[i]] ==2 or schema_id[list1[i]]==0]
label_idx = [i for i in range(len(list2)) if schema_id[list2[i]] ==2 or schema_id[list2[i]]==0]
pred_tokens = [batch['input_ids'][0][i] for i in pred_idx]
batch['token_ids'] = pred_tokens
print(tokenizer.decode(**batch))
label_tokens = [batch['input_ids'][0][i] for i in label_idx]
batch['token_ids'] = label_tokens
print(tokenizer.decode(**batch))

Assistant Professor Postctoral Researcher Research Research Associate
Assistant Professor Postdoctoral ResearcherMware Research Associate
